In [ ]:
"""
Author: Yang Hu
1. This file gives a script to optimise hyperparamters in GP in non-parametric analysis in this repository for a 
given BAO data file.
2. Constraints on parameters are obtained via Markov Chain Monte Carlo (MCMC) using emcee package.
"""

In [ ]:
"""
standard imports for data analysis; astropy.cosmology to compute astrophysical quantities with ease; george to run
GP and emcee to run MCMC
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import corner

import george
from george import kernels
from george.modeling import Model
import emcee

from astropy.cosmology import FlatLambdaCDM, FlatwCDM, LambdaCDM, wCDM

In [ ]:
"""
Load original data. Please check that the relevant data files are named and stored in the way the code below 
specified.
"""

#load&rename BAO data
data4 = pd.read_csv("Data/DESI_HZ_error.txt", delimiter=' ', skiprows=1, header=None) 
zBAO = data4[0] #x
sigHz = data4[1] #yerr

#generate y = Hz
H0_mock, Om0_mock, Ok0_mock, w_mock = 72, 0.3, 0.00, -1
truth = dict(h0=H0_mock, om=Om0_mock, ok=Ok0_mock, w=w_mock)
cosmo_mock = wCDM(H0=H0_mock, Om0=Om0_mock, Ode0=1.-Om0_mock-Ok0_mock, w0=w_mock)
Hz_mock = cosmo_mock.H(z=zBAO).value #y


In [ ]:
"""
Get a starting point of GP hyperparameters from BAO data using george package.
Constraints are obtained by MCMC and here we define the relevant prior and likelihood functions. 
"""

#compute a rough interpolation as starting values for MCMC
gp = george.GP(np.var(Hz_mock)*kernels.ExpSquaredKernel(1), fit_kernel=True,
               mean=np.mean(Hz_mock), white_noise=None)
print(gp.parameter_names)
gp.compute(zBAO, sigHz)

#defining mcmc functions
def lnprob(p):
    if np.any((-100 > p[1:]) + (p[1:] > 100)):
        return -np.inf
    gp.set_parameter_vector(p)
    return gp.log_likelihood(Hz_mock, quiet=True)

#Set up the sampler.
nwalkers, nsamples, ndim = 32, 20000, len(gp)
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

#Initialize the walkers.
p0 = gp.get_parameter_vector() + 1e-4 * np.random.randn(nwalkers, ndim)

In [ ]:
"""
run MCMC on hyperparameters using emcee package
"""

run_mcmc = True
save_mcmc = True
load_mcmc = True

labels = ["amp", "length scale"]

if run_mcmc:
    print("Sampling...")
    sampler.run_mcmc(p0, nsamples, progress=True);
    
if save_mcmc:
    samples = sampler.get_chain()
    flat_samples = sampler.get_chain(discard=5000, thin=4, flat=True)
    sample_data = pd.DataFrame(flat_samples)
    sample_data.to_csv(
    "GP_samples/amp_ls_%ix%i.csv" 
    % (nwalkers, nsamples), 
    index=False, header=labels
)

if load_mcmc:
    flat_samples = pd.read_csv("GP_samples/amp_ls_%ix%i.csv" % (nwalkers, nsamples), skiprows=1, header=None)

In [ ]:
"""
After loading a sample file, we can plot a corner plot showing how well constraints are placed on each interested
hyperparamters by the data.
"""

fig = corner.corner(
    flat_samples, labels=labels, quantiles=(0.16, 0.84), show_titles=True, 
    title_fmt='.3f', use_math_text=True
)
fig.suptitle("Corner plot, hyper-parameters: amp, length scale, %ix%i" % (nwalkers, nsamples), y=1.02)
fig.savefig("GP_plots/amp_ls_corner_%ix%i.png" % (nwalkers, nsamples), bbox_inches='tight')

In [ ]:
"""
plot interpolated curve with optimal hyperparameters for testing
"""

#here we need to replace the two numercial values by whatever values we get from MCMC
gp_optimal = george.GP(10.186*np.var(Hz_mock)*kernels.ExpSquaredKernel(1.939), fit_kernel=True,
               mean=np.mean(Hz_mock), white_noise=None)
gp_optimal.compute(zBAO, sigHz)

z_pred = np.linspace(0., max(zBAO), 500)
pred, pred_var = gp_optimal.predict(Hz_mock, z_pred, return_var=True)

plt.fill_between(z_pred, pred - np.sqrt(pred_var), pred + np.sqrt(pred_var),
                color="k", alpha=0.2)
plt.plot(z_pred, pred, "k", lw=1.5, alpha=0.5)
plt.errorbar(zBAO, Hz_mock, yerr=sigHz, fmt=".k", capsize=0)
plt.xlabel("$z_{BAO}$")
plt.ylabel("$H_z$")
plt.title("$H_z$ VS $z_{BAO}$, amp=10.186, length scale=1.939")
plt.savefig("GP_plots/optimal_hyperparameters.png")